## Data Processing

In [15]:
# Load packages
import pandas as pd
import numpy as np
import unittest 
import math
import glob
import os

In [25]:
# Clean and compile the data

# To Do: automate this
# Note: the truth column has been manually added to each ICA file for now

# Lie = 0 and Truth = 1

initialCSV = pd.concat(map(pd.read_csv, glob.glob(os.path.join('ICA', '*.csv'))), ignore_index = True)

for index, row in initialCSV.iterrows():
    if initialCSV.at[index, 'Truth'] == 0: initialCSV.at[index, 'Truth'] = -1
    
initialCSV

,EEG.AF3,EEG.T7,EEG.Pz,EEG.T8,EEG.AF4,Truth
0,5.57764,-18.68978,0.12047,-11.03442,13.27478,1
1,12.78836,-43.83118,0.75530,-25.16123,31.04901,1
2,12.26007,-44.01917,1.04022,-24.15714,30.72010,1
3,12.64842,-46.38778,0.76016,-25.72193,32.60343,1
4,15.47116,-55.28282,0.98514,-31.43475,39.30939,1
...,...,...,...,...,...,...
518395,1.28386,-0.26978,-0.13770,-0.49923,-0.40874,-1
518396,1.02784,-1.15967,0.04935,-0.17012,-0.45919,-1
518397,0.76818,-0.51442,0.05882,0.40821,0.45476,-1
518398,1.13579,0.75766,-0.66258,0.18601,1.03034,-1


In [36]:
# Partition the data into 80% training data and 20% testing data

initialCSV = initialCSV.head(50000)

num_rows = len(initialCSV.index)
first80p = math.floor(0.8 * num_rows)
last20p = num_rows - first80p

x_train = initialCSV.head(first80p).drop(columns = ['Truth'])
y_train = initialCSV.head(first80p)[['Truth']]

x_test = initialCSV.tail(last20p).drop(columns = ['Truth'])
y_test = initialCSV.tail(last20p)[['Truth']]

In [37]:
# Standardization

# Calculate avg. (mu) and stdev. (sigma) using the training set

d = x_train.shape[1]
mu = np.mean(x_train, axis = 0).values.reshape(1, d)
sigma = np.std(x_train, axis = 0).values.reshape(1, d)

# Transform the training features 

x_train = (x_train - mu) / (sigma + 1E-6)

# Transform the testing features

x_test = (x_test - mu) / (sigma + 1E-6)

print('Test Mean = ')
print(np.mean(x_test, axis = 0))

print('Test Standard Deviation = ')
print(np.std(x_test, axis = 0))

Test Mean = 
EEG.AF3   -0.056996
EEG.T7     0.088580
EEG.Pz     0.029744
EEG.T8     0.047421
EEG.AF4   -0.065909
dtype: float64
Test Standard Deviation = 
EEG.AF3    1.095160
EEG.T7     1.060452
EEG.Pz     1.338717
EEG.T8     1.111016
EEG.AF4    1.009386
dtype: float64


## Logistic Regression

The objective function is $Q(w; X, y) = \frac{1}{n} \sum_{i = 1}^n \log
\Big(1 + \exp \big(-y_i \; x_i^T \; w \big) \Big) + \frac{\lambda}{2} \|w\|_2^2$

When $\lambda = 0\;$, the model is typical logistic regression. When $\lambda > 0\;$, the model becomes regularized logistic regression.

In [28]:
# Calculate the value of the objective function (i.e. loss)

# Inputs: 
#          weight (w)   -->  d x 1 matrix
#          data (x)     -->  n x d matrix
#          label (y)    -->  n x 1 matrix
#          lmd (scalar) -->  regularization parameter

# Output: loss as a scalar

def objective(w, x , y, lmd):
    
    summ = 0.0
    n = len(x.index)
    
    for i in range(1, n + 1):
        
        xi = x.iloc[[n - 1]]
        yi = y.iloc[[n - 1]]
        
        expo = np.exp(-1 * np.dot(yi, np.dot(xi, w))[0][0])
        summ = summ + math.log(1 + expo, 10)
    
    return (summ / n) + 0.5 * lmd * np.square(w).sum() 

## Gradient Descent

The gradient at $w$ for regularized logistic regression is $g = - \frac{1}{n} \sum_{i = 1}^n \frac{y_i\;x_i}{1\;+\;\exp (\;y_i\;x_i^T\;w\;)} + \lambda\;w$

In [29]:
# Calculate the gradient at w

# Inputs: 
#          weight (w)   -->  d x 1 matrix
#          data (x)     -->  n x d matrix
#          label (y)    -->  n x 1 matrix
#          lmd (scalar) -->  regularization parameter

# Output: gradient (g) as a d x 1 matrix

def gradient(w, x, y, lmd):
    
    summ = 0.0
    n = len(x.index)
    
    for i in range(1, n + 1):
        
        # We want row i as a column
        xi = x.iloc[[n - 1]].T 
        yi = y.iloc[[n - 1]]
        
        expo = np.exp(np.dot(yi, np.dot(xi.T, w))[0][0])
        summ = summ + np.dot(xi, yi) / (1 + expo)
        
    return (lmd * w) - (summ / n)

In [30]:
# Gradient Descent for logistic regression
# Optimal weights will be obtained iteratively 

# Inputs: 
#          data (x)      -->  n x d matrix
#          label (y)     -->  n x 1 matrix
#          lmd (scalar)  -->  regularization parameter
#          learning_rate -->  scalar
#          weights (w)   -->  d x 1 matrix (initial)
#          max_epochs    -->  integer

# Outputs: 
#          weights (w)   -->  d x 1 matrix (final) 
#          objvals       -->  a record of each epoch's objective value 

def gradient_descent(x, y, lmd, learning_rate, w, max_epochs = 100):
    
    objvals = []
    
    for i in range(max_epochs):
        
        gt = gradient(w, x, y, lmd)
        w = w - learning_rate * gt
        objvals = objvals + [objective(w, x, y, lmd).iloc[0]]
    
    return w, objvals

## Training

Use Gradient Descent to obtain optimal weights and a list of objective values for each epoch

In [38]:
# Logistic Regression
# The number of data columns is 5

weights = np.random.randn(5, 1)
weights = pd.DataFrame(weights, columns = ['weights'])

logreg = gradient_descent(x_train, y_train, 0, 0.1, weights, 4)
logreg

(    weights
 0 -0.774123
 1 -0.442775
 2 -1.091699
 3  0.573863
 4 -0.193314,
 [0.1600760326228419,
  0.15488486555206285,
  0.14996900147987627,
  0.14530987089340527])

In [39]:
# Regularized Logistic Regression
# The number of data columns is 5

weightsR = np.random.randn(5, 1)
weightsR = pd.DataFrame(weightsR, columns = ['weights'])

reglogreg = gradient_descent(x_train, y_train, 0.5, 0.1, weightsR, 4)
reglogreg

(    weights
 0  0.244706
 1 -0.372956
 2  0.654179
 3  0.713414
 4 -0.621164,
 [0.6184332241769239,
  0.5862370211643857,
  0.5571697035920419,
  0.5309256227720796])

## Testing
Note: A smaller gap between training MSE and testing MSE indicates that the model generalizes better

In [42]:
# Predict the class label

# Inputs: 
#          weights (w)  -->  d x 1 matrix
#          data (X)     -->  m x d matrix

# Output: predictions (f) as an m x 1 matrix

def predict(w, X):
    return np.dot(X, w)

In [56]:
# Evaluate the training error

train_pred_GD_N = predict(logreg[0], x_train)
train_pred_GD_R = predict(reglogreg[0], x_train)

# [0] may need to be appended to these 2 lines when not indexing a scalar
train_MSE_GD_N = np.mean((train_pred_GD_N - y_train) ** 2)
train_MSE_GD_R = np.mean((train_pred_GD_R - y_train) ** 2)

print('LogReg:\t\t', train_MSE_GD_N)
print('RegLogReg:\t', train_MSE_GD_R) 

LogReg:		 3.519333821553146
RegLogReg:	 2.1807175319428005


In [57]:
# Evaluate the testing error

test_pred_GD_N = predict(logreg[0], x_test)
test_pred_GD_R = predict(reglogreg[0], x_test)

# [0] may need to be appended to these 2 lines when not indexing a scalar
test_MSE_GD_N = np.mean((test_pred_GD_N - y_test) ** 2)
test_MSE_GD_R = np.mean((test_pred_GD_R - y_test) ** 2)

print('LogReg:\t\t', test_MSE_GD_N)
print('RegLogReg:\t', test_MSE_GD_R) 

LogReg:		 4.328379185946702
RegLogReg:	 2.769418847480221


In [130]:
# Calculate accuracy

def acc(predict, actual):
    
    correct = 0
    rounded = list(map(lambda x: 1 if x > 0 else -1, predict))
    
    for i in range(len(rounded)): 
        correct = correct + 1 if rounded[i] == actual.iloc[i].iloc[0] else correct
    
    return correct / len(rounded)

In [131]:
# Evaluate training accuracy

acc_train_N = acc(train_pred_GD_N, y_train)
acc_train_R = acc(train_pred_GD_R, y_train)

print('LogReg:\t\t', acc_train_N)
print('RegLogReg:\t', acc_train_R) 

LogReg:		 0.4956
RegLogReg:	 0.503


In [133]:
# Evaluate testing accuracy

acc_test_N = acc(test_pred_GD_N, y_test)
acc_test_R = acc(test_pred_GD_R, y_test)

print('LogReg:\t\t', acc_test_N)
print('RegLogReg:\t', acc_test_R) 

LogReg:		 0.5071
RegLogReg:	 0.5017


## Unit Testing

In [136]:
class TestModel(unittest.TestCase):
    
    def test_muNotEmpty(self):
        self.assertIsNotNone(mu)
        
    def test_sigNotEmpty(self):
        self.assertIsNotNone(sigma)
        
    def test_weightsNotEmpty(self):
        self.assertIsNotNone(weights)
    
    def test_weightsRNotEmpty(self):
        self.assertIsNotNone(weightsR)
    
    def test_MSE_N_train(self):
        self.assertGreater(train_MSE_GD_N, 0)
        self.assertLess(train_MSE_GD_N, 100)
            
    def test_MSE_R_train(self):
        self.assertGreater(train_MSE_GD_R, 0)
        self.assertLess(train_MSE_GD_R, 100)
    
    def test_MSE_N_test(self):
        self.assertGreater(test_MSE_GD_N, 0)
        self.assertLess(test_MSE_GD_N, 100)
    
    def test_MSE_R_test(self):
        self.assertGreater(test_MSE_GD_R, 0)
        self.assertLess(test_MSE_GD_R, 100)
            
    def test_acc_N_test(self):
        self.assertGreater(acc_test_N, 0.50)
        
    def test_acc_R_test(self):
        self.assertGreater(acc_test_R, 0.50)

unittest.main(argv = [''], verbosity = 2, exit = False)

test_MSE_N_test (__main__.TestModel) ... ok
test_MSE_N_train (__main__.TestModel) ... ok
test_MSE_R_test (__main__.TestModel) ... ok
test_MSE_R_train (__main__.TestModel) ... ok
test_acc_N_test (__main__.TestModel) ... ok
test_acc_R_test (__main__.TestModel) ... ok
test_muNotEmpty (__main__.TestModel) ... ok
test_sigNotEmpty (__main__.TestModel) ... ok
test_weightsNotEmpty (__main__.TestModel) ... ok
test_weightsRNotEmpty (__main__.TestModel) ... ok

----------------------------------------------------------------------
Ran 10 tests in 0.028s

OK
